In [967]:
import pandas as pd
import re
import regex
import emoji
import numpy as np
import demoji
from collections import Counter
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import streamlit as st
import plotly.express as px

1.- Definir el documento a tablas 

In [968]:

ruta = r"C:\Users\carlo\OneDrive\Desktop\analisis mensajes con mi novia"



with open("_chat.txt", "r", encoding="utf-8") as file:
    lineas = file.readlines()


patron = r"\[(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2}:\d{2})\] (.*?): (.*)"

data = []

for lineas in lineas:
    match = re.match(patron, lineas)
    if match:
        fecha, hora, usuario, mensaje = match.groups()
        data.append([fecha, hora, usuario, mensaje])
    
df = pd.DataFrame(data, columns=["Fecha", "Hora", "Usuario", "Mensaje"])

df["Hora_dt"] = pd.to_datetime(df["Hora"], format="%H:%M:%S")
df["Hora_24"] = df["Hora_dt"].dt.hour
df["Horario"] = df["Hora_dt"].dt.strftime("%I %p").str.lstrip("0")
df = df[~df["Mensaje"].str.contains("end-to-end encrypted", na=False)]



In [969]:
df['te_amo'] = df['Mensaje'].str.lower().str.count(r'\bte amo\b')
te_amo_por_usuario = df.groupby('Usuario')['te_amo'].sum().reset_index()
te_amo_por_usuario = te_amo_por_usuario.sort_values(by='te_amo', ascending=False)
te_amo_por_usuario

,Usuario,te_amo
0,Carlos Antonio,119
1,Danita 🩷,65


In [970]:
import plotly.express as px
colores_rojos = ['#FF6666', '#FF3333', '#CC0000', '#990000']

fig = px.pie(
    te_amo_por_usuario,
    names='Usuario',       
    values='te_amo',       
    color='Usuario',       
    color_discrete_sequence=colores_rojos
)

fig.update_traces(textposition='inside', textinfo='percent+label', 
                  hovertemplate='<b>%{label}</b><br>Cantidad: %{value}<br>Porcentaje: %{percent}')

fig.update_layout(title='Distribución interactiva de "te amo" por usuario')
st.plotly_chart(fig, use_container_width=True)

2026-02-21 10:38:09.470 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-02-21 10:38:09.475 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.476 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.478 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.479 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.480 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [971]:
mensajes_por_hora = df.groupby('Horario')[['Mensaje']].count()

mensajes_por_hora = mensajes_por_hora.rename(columns={'Hora_12' : 'Horario', 'Mensaje' : 'Cantidad'})
mensajes_por_hora = mensajes_por_hora.sort_values(by = "Cantidad", ascending=False)

top_3 = mensajes_por_hora.head(3)
top_3 = top_3.reset_index()
top_3

,Horario,Cantidad
0,9 PM,1454
1,8 PM,1038
2,7 PM,1035


In [972]:

colores_rojos = ['#FF6666', '#FF3333', '#CC0000', '#990000']

fig = px.pie(
    top_3,
    names='Horario',       
    values='Cantidad',       
    color='Horario',       
    color_discrete_sequence=colores_rojos
)

fig.update_traces(textposition='inside', textinfo='percent+label', 
                  hovertemplate='<b>%{label}</b><br>Cantidad: %{value}<br>Porcentaje: %{percent}')

fig.update_layout(title='Distribución interactiva de "te amo" por usuario')

st.plotly_chart(fig, use_container_width=True)

2026-02-21 10:38:09.540 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.


2026-02-21 10:38:09.542 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.543 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.544 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:09.544 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [973]:
df['Palabras_usasdas_por_mensaje'] = df['Mensaje'].str.split().str.len()

In [974]:
total_palabras = ' '
stopwords = STOPWORDS.update(['que', 'qué', 'con', 'de', 'te', 'en', 'la', 'lo', 'le', 'el', 'las', 'los', 'les', 
                              'por', 'es', 'son', 'se', 'para', 'un', 'una', 'chicos', 'su', 'si', 'chic','nos', 'ya', 
                              'hay', 'esta', 'pero', 'del', 'mas', 'más', 'eso', 'este', 'como', 'así', 'todo', 
                              'https','Media','omitted','y', 'mi', 'o', 'q', 'yo', 'al'])

mask = np.array(Image.open('heart.jpg'))


for mensaje in df['Mensaje'].values:
    palabras = str(mensaje).lower().split()
    for palabra in palabras:
        total_palabras += palabra + ' '


wordcloud = WordCloud(
    width=800,
    height=800,
    background_color='black',
    stopwords=stopwords,
    max_words=100,
    min_font_size=5,
    mask=mask,
    colormap='OrRd'
).generate(total_palabras)


imagen_wc = wordcloud.to_image()

st.image(imagen_wc, use_column_width=True)

2026-02-21 10:38:17.042 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.043 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.044 The `use_column_width` parameter has been deprecated and will be removed in a future release. Please utilize the `width` parameter instead.
2026-02-21 10:38:17.045 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.052 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.053 Thread 'MainThread': missing ScriptRunContext! This warn

DeltaGenerator()

In [975]:
import emoji

def ObtenerEmojis(texto):
    return [e['emoji'] for e in emoji.emoji_list(str(texto))]

# Crear columna nueva
df['Emoji_list'] = df['Mensaje'].apply(ObtenerEmojis)

grouped = df.groupby('Usuario').agg(
    Mensajes=('Mensaje', 'count'),
    Total_Emojis=('Emoji_list', lambda x: sum(len(i) for i in x)),
    Total_Palabras=('Palabras_usasdas_por_mensaje', 'sum'),
    Promedio_palabras=('Palabras_usasdas_por_mensaje', 'mean')
).reset_index()



In [976]:

total_mensajes = grouped['Mensajes'].sum()
grouped['%Mensajes'] = (grouped['Mensajes'] / total_mensajes) * 100
grouped['%Mensajes'] = grouped['%Mensajes'].round(2)

grouped

,Usuario,Mensajes,Total_Emojis,Total_Palabras,Promedio_palabras,%Mensajes
0,Carlos Antonio,6292,225,34181,5.432454,52.11
1,Danita 🩷,5782,224,31449,5.439121,47.89


In [977]:
emojis_lista = list([a for b in df.Emoji_list for a in b])
emoji_diccionario = dict(Counter(emojis_lista))
emoji_diccionario = sorted(emoji_diccionario.items(), key=lambda x: x[1], reverse=True)


emoji_df = pd.DataFrame(emoji_diccionario, columns=['Emoji', 'Cantidad'])

emoji_df = pd.DataFrame(emoji_diccionario, columns=['Emoji', 'Cantidad'])

print('Número emojis únicos usados: ', len(emoji_df), '\n')
emoji_df.head(10)

Número emojis únicos usados:  68 



,Emoji,Cantidad
0,❤️,184
1,🧡,49
2,🩷,33
3,☺️,21
4,🥹,15
5,😍,13
6,💕,13
7,😭,10
8,🫶🏽,8
9,😱,7


In [978]:
colores_rojos = ['#FF6666', '#FF3333', '#CC0000', '#990000']

fig = px.pie(
    emoji_df,
    names='Emoji',       
    values='Cantidad',       
    color='Emoji',       
    color_discrete_sequence=colores_rojos
)

fig.update_traces(textposition='inside', textinfo='percent+label', 
                  hovertemplate='<b>%{label}</b><br>Cantidad: %{value}<br>Porcentaje: %{percent}')

fig.update_layout(title='Distribución interactiva de "te amo" por usuario')

st.plotly_chart(fig, use_container_width=True)

2026-02-21 10:38:17.383 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-02-21 10:38:17.385 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.388 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.389 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [979]:
df['rangoHora'] = pd.to_datetime(df['Hora'], format='%H:%M:%S')


def create_range_hour(hour):
    hour = pd.to_datetime(hour)  
    start_hour = hour.hour
    end_hour = (hour + pd.Timedelta(hours=1)).hour
    return f'{start_hour:02d} - {end_hour:02d} h'


df['rangoHora'] = df['rangoHora'].apply(create_range_hour)
df = pd.DataFrame({
    'FECHA' : df['Fecha'],
    'usuario' : df['Usuario'],
    'HORA' : df['Horario'],
    'Rango Hora' : df['rangoHora']

})


In [980]:
df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')  
df['DiaSemana'] = df['FECHA'].dt.strftime('%A')
mapeo_dias_espanol = {'Monday': '1 Lunes','Tuesday': '2 Martes','Wednesday': '3 Miércoles','Thursday': '4 Jueves',
                      'Friday': '5 Viernes','Saturday': '6 Sábado','Sunday': '7 Domingo'}
df['DiaSemana'] = df['DiaSemana'].map(mapeo_dias_espanol)
df 

C:\Users\carlo\AppData\Local\Temp\ipykernel_7864\4185808133.py:1: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



,FECHA,usuario,HORA,Rango Hora,DiaSemana
1,2025-11-21,Carlos Antonio,9 AM,09 - 10 h,5 Viernes
2,2025-11-21,Carlos Antonio,9 AM,09 - 10 h,5 Viernes
3,2025-11-21,Danita 🩷,10 AM,10 - 11 h,5 Viernes
4,2025-11-21,Carlos Antonio,10 AM,10 - 11 h,5 Viernes
5,2025-11-21,Danita 🩷,10 AM,10 - 11 h,5 Viernes
...,...,...,...,...,...
12070,2026-02-20,Danita 🩷,9 AM,09 - 10 h,5 Viernes
12071,2026-02-20,Carlos Antonio,9 AM,09 - 10 h,5 Viernes
12072,2026-02-20,Danita 🩷,9 AM,09 - 10 h,5 Viernes
12073,2026-02-20,Danita 🩷,9 AM,09 - 10 h,5 Viernes


In [981]:

df['# Mensajes por hora'] = 1


mensajes_hora = df.groupby('Rango Hora').count().reset_index()


fig = px.line(mensajes_hora, x='Rango Hora', y='# Mensajes por hora', color_discrete_sequence=['salmon'], template='plotly_dark')


fig.update_layout(
    title={'text': 'Mensajes por hora', 'y':0.96, 'x':0.5, 'xanchor': 'center'},
    font=dict(size=17))
fig.update_traces(mode='markers+lines', marker=dict(size=10))
fig.update_xaxes(title_text='Rango de hora', tickangle=30)
fig.update_yaxes(title_text='# Mensajes')
st.plotly_chart(fig, use_container_width=True)

2026-02-21 10:38:17.715 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-02-21 10:38:17.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.718 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.719 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.720 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [982]:

df['# Mensajes por día'] = 1

date_df = df.groupby('DiaSemana').count().reset_index()


fig = px.line(date_df, x='DiaSemana', y='# Mensajes por día', color_discrete_sequence=['salmon'], template='plotly_dark')


fig.update_layout(
    title={'text': 'Mensajes que tenemos por dia en la semana', 'y':0.96, 'x':0.5, 'xanchor': 'center'},
    font=dict(size=17))
fig.update_traces(mode='markers+lines', marker=dict(size=10))
fig.update_xaxes(title_text='Día', tickangle=30)
fig.update_yaxes(title_text='# Mensajes')
st.plotly_chart(fig, use_container_width=True)

2026-02-21 10:38:17.769 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-02-21 10:38:17.772 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.773 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.774 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [983]:

df['# Mensajes por día'] = 1

date_df = df.groupby('FECHA')['# Mensajes por día'].sum().reset_index()

fig = px.line(
    date_df,
    x='FECHA',
    y='# Mensajes por día',
    color_discrete_sequence=['salmon'],
    template='plotly_dark',
    title='Mensajes por día '
)

st.plotly_chart(fig, use_container_width=True)

2026-02-21 10:38:17.822 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2026-02-21 10:38:17.825 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.826 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.827 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.828 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-21 10:38:17.829 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()